In [ ]:
import numpy as np
import cv2
from collections import namedtuple
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import igraph as ig
import logging
import sys
from tqdm.auto import tqdm as tq

In [ ]:
con = namedtuple('_', ('FIX', 'UNK', 'FG', 'BG'))(1, 0, 1, 0)
NUM_GMM_COMP = 5
GAMMA = 50
LAMDA = 9 * GAMMA
NUM_ITERS = 3
TOL = 1e-3

In [ ]:
class EventHandler:
    """
    Class for handling user input during segmentation iterations 
    """
    
    def __init__(self, flags, img, _types, _alphas, colors):
        
        self.FLAGS = flags
        self.ix = -1
        self.iy = -1
        self.img = img
        self.img2 = self.img.copy()
        self._types = _types
        self._alphas = _alphas
        self.COLORS = colors

    @property
    def image(self):
        return self.img
    
    @image.setter
    def image(self, img):
        self.img = img
        
    @property
    def types(self):
        return self._types

    @types.setter
    def types(self, _types):
        self._types = _types
    
    @property
    def alphas(self):
        return self._alphas

    @alphas.setter
    def alphas(self, _alphas):
        self._alphas = _alphas
    
    @property
    def flags(self):
        return self.FLAGS 
    
    @flags.setter
    def flags(self, flags):
        self.FLAGS = flags
    
    def handler(self, event, x, y, flags, param):

        # Draw the rectangle first
        if event == cv2.EVENT_LBUTTONDOWN:
            if self.FLAGS['rect_over'] == False:
                self.FLAGS['DRAW_RECT'] = True
                self.ix, self.iy = x,y

        elif event == cv2.EVENT_MOUSEMOVE:
            if self.FLAGS['DRAW_RECT'] == True:
                self.img = self.img2.copy()
                cv2.rectangle(self.img, (self.ix, self.iy), (x, y), self.COLORS['BLUE'], 2)
                self.FLAGS['RECT'] = (min(self.ix, x), min(self.iy, y), abs(self.ix - x), abs(self.iy - y))
                self.FLAGS['rect_or_mask'] = 0

        elif event == cv2.EVENT_LBUTTONUP:
            if self.FLAGS['rect_over'] == False:
                self.FLAGS['DRAW_RECT'] = False
                self.FLAGS['rect_over'] = True
                cv2.rectangle(self.img, (self.ix, self.iy), (x, y), self.COLORS['BLUE'], 2)
                self.FLAGS['RECT'] = (min(self.ix, x), min(self.iy, y), abs(self.ix - x), abs(self.iy - y))
                self.FLAGS['rect_or_mask'] = 0

                # initialise types and alphas
                temp = np.zeros(self._types.shape, dtype=np.uint8)
                rect = self.FLAGS['RECT']
                temp[rect[1]:rect[1]+rect[3], rect[0]:rect[0]+rect[2]] = 1
                self._types[temp == 0] = con.FIX
                self._alphas[rect[1]:rect[1]+rect[3], rect[0]:rect[0]+rect[2]] = con.FG

        
        # Draw strokes for refinement 

        if event == cv2.EVENT_LBUTTONDOWN:
            if self.FLAGS['rect_over'] == True:
                self.FLAGS['DRAW_STROKE'] = True
                cv2.circle(self.img, (x,y), 3, self.FLAGS['value']['color'], -1)
                cv2.circle(self._alphas, (x,y), 3, self.FLAGS['value']['val'], -1)
                cv2.circle(self._types, (x,y), 3, con.FIX, -1)

        elif event == cv2.EVENT_MOUSEMOVE:
            if self.FLAGS['DRAW_STROKE'] == True:
                cv2.circle(self.img, (x, y), 3, self.FLAGS['value']['color'], -1)
                cv2.circle(self._alphas, (x,y), 3, self.FLAGS['value']['val'], -1)
                cv2.circle(self._types, (x,y), 3, con.FIX, -1)

        elif event == cv2.EVENT_LBUTTONUP:
            if self.FLAGS['DRAW_STROKE'] == True:
                self.FLAGS['DRAW_STROKE'] = False
                cv2.circle(self.img, (x, y), 3, self.FLAGS['value']['color'], -1)
                cv2.circle(self._alphas, (x,y), 3, self.FLAGS['value']['val'], -1)
                cv2.circle(self._types, (x,y), 3, con.FIX, -1)

In [ ]:
def fit_gmms(img, alphas, n_components):
    fg = GaussianMixture(n_components=n_components)
    fg.fit(img[alphas == con.FG].reshape((-1, img.shape[-1])))

    bg = GaussianMixture(n_components=n_components)
    bg.fit(img[alphas == con.BG].reshape((-1, img.shape[-1])))

    return fg, bg

def graph_cut(img, types, alphas, fg_gmm, bg_gmm, beta, gamma, lamda, connect_diag):
    logging.info('GRAPH CUT')
    
    # compute region energy in one go to speed up
    fg_D = - fg_gmm.score_samples(img.reshape((-1, img.shape[-1]))).reshape(img.shape[:-1])
    bg_D = - bg_gmm.score_samples(img.reshape((-1, img.shape[-1]))).reshape(img.shape[:-1])

    # closure function to calculate boundary energy
    def compute_V(i, j, oi, oj):
        diff = img[i, j] - img[oi, oj]
        return gamma * np.exp(- beta * diff.dot(diff))

    # fixed capacity for known edges
    fix_cap = lamda

    # BUILD GRAPH
    logging.info('BUILD GRAPH')
    num_pix = img.shape[0] * img.shape[1]

    def vid(i, j): # vertex ID
        return (img.shape[1] * i) + j

    def ind(idx): # image index
        return ((idx // img.shape[1]), (idx % img.shape[1]))
    
    graph = ig.Graph(directed=False)
    graph.add_vertices(num_pix + 2)
    S = num_pix
    T = num_pix+1
    # the last two vertices are S and T respectively

    edges = []
    weights = []
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):

            # add edges to S and T
            if types[i, j] == con.FIX:
                if alphas[i, j] == con.FG:
                    edges.append((vid(i, j), S))
                    weights.append(fix_cap)
                else:
                    edges.append((vid(i, j), T))
                    weights.append(fix_cap)
            else:
                edges.append((vid(i, j), S))
                weights.append(bg_D[i, j])

                edges.append((vid(i, j), T))
                weights.append(fg_D[i, j])
            
            # add edges to neighbours
            if i > 0:
                oi = i-1
                oj = j
                edges.append((vid(i, j), vid(oi, oj)))
                weights.append(compute_V(i, j, oi, oj))
            
            if j > 0:
                oi = i
                oj = j-1 
                edges.append((vid(i, j), vid(oi, oj)))
                weights.append(compute_V(i, j, oi, oj))

            if connect_diag:
                if i > 0 and j > 0:
                    oi = i-1
                    oj = j-1 
                    edges.append((vid(i, j), vid(oi, oj)))
                    weights.append(compute_V(i, j, oi, oj))

                if i > 0 and j < img.shape[1] - 1:
                    oi = i-1
                    oj = j+1 
                    edges.append((vid(i, j), vid(oi, oj)))
                    weights.append(compute_V(i, j, oi, oj))
    
    graph.add_edges(edges, attributes={'weight': weights})
    logging.info('MINCUT')
    cut = graph.st_mincut(S, T, capacity='weight')
    bg_vertices = cut.partition[0]
    fg_vertices = cut.partition[1]
    if S in bg_vertices:
        bg_vertices, fg_vertices = fg_vertices, bg_vertices
    
    new_alphas = np.zeros(img.shape[:2], dtype=np.uint8) # con.BG is filled, zeroes is faster
    for v in fg_vertices:
        if v not in (S, T):
            new_alphas[ind(v)] = 1
    return cut.value, new_alphas


def grab_cut(img_, types_, alphas_, n_components, gamma, lamda,
             num_iters, tol, connect_diag):
    
    logging.debug('GRAB CUT')
    img = img_.copy().astype(np.float32)
    types = types_.copy() # types tells whether the pixel is fixed or unknown
    alphas = alphas_.copy() # alphas tells whether the pixel is fg or bg according to fixed or current estimate
    
    # calculate beta
    logging.info('CALC BETA')
    beta = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if i > 0:
                diff = img[i, j] - img[i-1, j]
                beta += diff.dot(diff)
            if j > 0:
                diff = img[i, j] - img[i, j-1]
                beta += diff.dot(diff)
            if connect_diag:
                if i > 0 and j > 0:
                    diff = img[i, j] - img[i-1, j-1]
                    beta += diff.dot(diff)
                if i > 0 and j < img.shape[1] - 1:
                    diff = img[i, j] - img[i-1, j+1]
                    beta += diff.dot(diff)
    if connect_diag:
        beta /= (4 * img.shape[0] * img.shape[1] - 3 * img.shape[0] - 3 * img.shape[1] + 2)
    else:
        beta /= (2 * img.shape[0] * img.shape[1] - img.shape[0] - img.shape[1])
    beta *= 2
    beta = 1 / beta
    
    prev_flow = -1
    for _ in tq(range(num_iters)):
        fg_gmm, bg_gmm = fit_gmms(img, alphas, n_components)
        flow, alphas = graph_cut(img, types, alphas, fg_gmm, bg_gmm, beta, gamma, lamda, connect_diag)
    
        if prev_flow != -1 and abs(prev_flow - flow) < tol:
            break
        
        prev_flow = flow
    
    logging.info('DONE')
    return alphas

In [ ]:
def run(filename, n_components=NUM_GMM_COMP, gamma=GAMMA, lamda=LAMDA,
        num_iters=NUM_ITERS, tol=TOL, connect_diag=True):
    """
    Main loop that implements GrabCut. 
    
    Input
    -----
    filename (str) : Path to image
    """
    
    COLORS = {
    'BLACK' : [0,0,0],
    'RED'   : [0, 0, 255],
    'GREEN' : [0, 255, 0],
    'BLUE'  : [255, 0, 0],
    'WHITE' : [255,255,255]
    }

    DRAW_BG = {'color' : COLORS['BLACK'], 'val' : con.BG}
    DRAW_FG = {'color' : COLORS['WHITE'], 'val' : con.FG}

    FLAGS = {
        'RECT' : (0, 0, 1, 1),
        'DRAW_STROKE': False,         # flag for drawing strokes
        'DRAW_RECT' : False,          # flag for drawing rectangle
        'rect_over' : False,          # flag to check if rectangle is  drawn
        'rect_or_mask' : -1,          # flag for selecting rectangle or stroke mode
        'value' : DRAW_FG,            # drawing strokes initialized to mark foreground
    }

    img = cv2.imread(filename)
    img2 = img.copy()
    types = np.zeros(img.shape[:2], dtype = np.uint8)  # whether a pixel is known or unknown
    alphas = np.zeros(img.shape[:2], dtype = np.uint8) # mask is a binary array with : 0 - background pixels
                                                       #                               1 - foreground pixels 
    output = np.zeros(img.shape, np.uint8)             # output image to be shown

    # Input and segmentation windows
    cv2.namedWindow('Input Image')
    cv2.namedWindow('Segmented output')
    
    EventObj = EventHandler(FLAGS, img, types, alphas, COLORS)
    cv2.setMouseCallback('Input Image', EventObj.handler)
    cv2.moveWindow('Input Image', img.shape[1] + 10, 90)

    while(1):
        
        img = EventObj.image
        types = EventObj.types
        alphas = EventObj.alphas
        FLAGS = EventObj.flags
        cv2.imshow('Segmented image', output)
        cv2.imshow('Input Image', img)
        
        k = cv2.waitKey(1)

        # key bindings
        if k == 27:
            # esc to exit
            break
        
        elif k == ord('0'): 
            # Strokes for background
            FLAGS['value'] = DRAW_BG
        
        elif k == ord('1'):
            # FG drawing
            FLAGS['value'] = DRAW_FG
        
        elif k == ord('r'):
            # reset everything
            FLAGS['RECT'] = (0, 0, 1, 1)
            FLAGS['DRAW_STROKE'] = False
            FLAGS['DRAW_RECT'] = False
            FLAGS['rect_or_mask'] = -1
            FLAGS['rect_over'] = False
            FLAGS['value'] = DRAW_FG
            img = img2.copy()
            types = np.zeros(img.shape[:2], dtype = np.uint8) 
            alphas = np.zeros(img.shape[:2], dtype = np.uint8)
            EventObj.image = img
            EventObj.types = types
            EventObj.alphas = alphas
            output = np.zeros(img.shape, np.uint8)
        
        elif k == 13: 
            # Press carriage return to initiate segmentation
            
            #-------------------------------------------------#
            # Implement GrabCut here.                         #  
            # Function should return a mask which can be used #
            # to segment the original image as shown on L90   # 
            #-------------------------------------------------#
            alphas = grab_cut(img2, types, alphas, n_components, gamma, lamda, num_iters, tol, connect_diag)
            EventObj.alphas = alphas

        
        EventObj.flags = FLAGS
        mask2 = np.where((alphas == 1), 255, 0).astype('uint8')
        output = cv2.bitwise_and(img2, img2, mask = mask2)

In [ ]:
filename = 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\png\\airplane.png'              # Path to image file
try:
    run(filename)
finally:
    cv2.destroyAllWindows()